# Отчет по Лабоработной работе №2

## 1. Запуск _Graph500_ на локальной машине

Собрать и запустить _Graph500_ на локально машине (MacOsX) не получилось. Однако, я завел равную по мощности виртуальную машину в облаке и собрал\протестировал на ней.

Для сборки я написал скрипт [init.sh](init.sh), который устанавливает необходимые зависимости для сборки\запуска теста на linux-like машинку, а так же собирает бинари для теста.

**Конфигурация машинки**:
  * __Name__:         mega-graph500
  * __Тип инстанса__: c1.xlarge
  * __RAM__:          16ГБ
  * __VCPUs__:        16 ВЦПУ
  * __Диск__:         40ГБ
  * __OS__:           Ubuntu 14.04 

### Запуск в 1 MPI поток

**mega-graph500**```$> graph500-2.1.4/mpi/graph500_mpi_simple 16``` `2>` [cerr_16_16_only_mega-graph500.txt](cerr_16_16_only_mega-graph500.txt)

```
...
min_TEPS:                       5.55614e+07
firstquartile_TEPS:             5.93069e+07
median_TEPS:                    6.05042e+07
thirdquartile_TEPS:             6.58648e+07
max_TEPS:                       6.98376e+07
harmonic_mean_TEPS:             6.17377e+07
harmonic_stddev_TEPS:           450093
...
```
**Полная статистика тут:** [cout_16_16_only_mega-graph500.txt](cout_16_16_only_mega-graph500.txt)

Замеряная производительность равна примерно **0.061 GTEPS**, что соответствует **186** месту в [рейтинге Graph500](http://www.graph500.org/results_jul_2015)

### Запуск в 16 MPI потоков

Так же было замеряна статистика при запуске на этой машине теста через MPI в 16 потоков (по числу ядер):

**mega-graph500**```$> mpirun -np 16 graph500-2.1.4/mpi/graph500_mpi_simple 16``` `2>` [cerr_n16_16_16_only_mega-graph500.txt](cerr_n16_16_16_only_mega-graph500.txt)

```
...
min_TEPS:                       1.19341e+07
firstquartile_TEPS:             1.38494e+08
median_TEPS:                    1.83064e+08
thirdquartile_TEPS:             1.87837e+08
max_TEPS:                       1.92238e+08
harmonic_mean_TEPS:             7.90478e+07
harmonic_stddev_TEPS:           1.5918e+07
...
```
**Полная статистика тут:** [cout_n16_16_16_only_mega-graph500.txt](cout_n16_16_16_only_mega-graph500.txt)

Замеряная производительность равна примерно предыдущей, но в пике тест выдавал до **0.192 GTEPS**, что соответствует **182** месту в [рейтинге Graph500](http://www.graph500.org/results_jul_2015)

## 2. Запуск _Graph500_ на кластере

Под рукой кластера не оказалось, но я решил собрать свой кластер из слабых виртуальных машин. Минус данного подхода в том, что сеть между данными машинами очень медленная, что ухудшает производительность MPI приложений (а, соответственно, и результаты теста получаются хуже).

**Конфигурация машинок**:
  * __Name__:         graph500-{1..8}
  * __Тип инстанса__: m1.tiny
  * __RAM__:          1ГБ
  * __VCPUs__:        1 ВЦПУ
  * __Диск__:         2ГБ
  * __OS__:           Ubuntu 14.04 

Для автоматизации предворительной настройки использоватлся скрипт [deploy.sh](deploy.sh), 
который копировал [init.sh](init.sh) скрипт на все машинки и запускал его там.

Перед запуском вычислений нужно было настроить и проверить связность по ssh между головной машиной (__graph500-1__) и остальным кластером.
Для этого использовался скрипт [test.sh](test.sh)

### Запуск в 1 MPI поток на одной машине

**graph500-1**```$> graph500-2.1.4/mpi/graph500_mpi_simple 16``` `2>` [cerr_16_16_only_graph500-1.txt](cerr_16_16_only_graph500-1.txt)

```
...
min_TEPS:                       6.56382e+07
firstquartile_TEPS:             7.01803e+07
median_TEPS:                    7.15393e+07
thirdquartile_TEPS:             7.4955e+07
max_TEPS:                       8.65122e+07
harmonic_mean_TEPS:             7.23425e+07
harmonic_stddev_TEPS:           486762
...
```
**Полная статистика тут:** [cout_16_16_only_graph500-1.txt](cout_16_16_only_graph500-1.txt)

Как ни странно, на машинке с такими слабыми характеристиками, тест показал результат в **0.072 GTEPS**, что выше чем при аналогичном запуске на машине [mega-graph500](#Запуск-в-1-MPI-поток). Но в [рейтинге]((http://www.graph500.org/results_jul_2015) это все еще **186** место.

### Запуск в 8 MPI потоков на кластере (по 1 потоку на каждой машинке)

**graph500-1**```$> mpirun -n 8 -f ``` [hostfile](hostfile) ```graph500-2.1.4/mpi/graph500_mpi_simple 16``` `2>` [cerr_n8_16_16_from_graph500-1.txt](cerr_n8_16_16_from_graph500-1.txt)

```
...
min_TEPS:                       1.3583e+07
firstquartile_TEPS:             1.66226e+07
median_TEPS:                    1.81347e+07
thirdquartile_TEPS:             1.889e+07
max_TEPS:                       2.23661e+07
harmonic_mean_TEPS:             1.74875e+07
harmonic_stddev_TEPS:           244649
...
```
**Полная статистика тут:** [cout_n8_16_16_from_graph500-1.txt](cout_n8_16_16_from_graph500-1.txt)

Как и ожидалось, результаты в несколько раз хуже чем аналогичный однопоточный результат - **0.018 GTEPS**. Объяснить такой результат можно накладными расходами на передачу сообщений по сети. В [рейтинге]((http://www.graph500.org/results_jul_2015) данный показатель занимает **последнее место**.